## Building a Chat Bot

In the examples below, we will build a very simple application that creates a simple chat bot. The bot is monitoring the actions that are happening on Slack, and posts some acknowledgement messages every time someone types a message.


#### Authentication

We start first by authenticating our application. (See notes in S1). We read the `slack_secret.json` file, and we instantiate then our SlackClient.

In [ ]:
# Read the access token from the file
import json

secrets_file = 'slack_secret.json'
f = open(secrets_file, 'r') 
content = f.read()
f.close()

auth_info = json.loads(content)
token = auth_info["access_token"]

In [ ]:
# If you happen not to have the slackclient library installed, 
# you can uncomment and run the code below
!sudo -H python3 -m pip install -U slackclient

In [ ]:
from slackclient import SlackClient
sc = SlackClient(token)
response = sc.api_call("users.info", user=auth_info["user_id"])
user = response['user']
print("The username of the authenticated user is", user.get('name'))
print("The email of the authenticated user is", user.get('profile').get('email'))
print("The email of the authenticated user is", user.get('profile').get('real_name'))

### Implementing the bot

The code below continuously monitors the various events on Slack. Of course, the bot can be programmed to react to any type of event. In this particular example, the bot monitors for user messages, and posts a "thank you" note to each user into the #bots channel.

In [ ]:
import time
import re

if sc.rtm_connect():
    # We are going to be polling the Slack API for recent events continuously
    while True:
        # We are going to wait 1 second between monitoring attempts
        time.sleep(1)
        # If there are any new events, we will get a response. If there are no events, the response will be empty
        response = sc.rtm_read()
        for item in response:
            event_type = item.get("type")
            # If the event is a message and the message is written by a user (and not a bot)
            if event_type == 'message' and item.get("user")!=None:
                print(item)
                print("=========================")
                message = "Thank you user {u} for participating in channel {c}".format(u=item["user"], c=item["channel"])
                sc.api_call("chat.postMessage", channel="#bots", text=message)

#### Exercises 

* Change the code above to retrieve the actual user name of the user, instead of the user id
* Chagne the code above to retrieve the actual name of the channel instead of the channel id
* Change the code above to post the answer to the channel where the user posted, instead of the '#bots' channel